In [1]:
# Installing Gradio
!pip install gradio transformers -q

# Import the required Libraries
import gradio as gr
import numpy as np
import pandas as pd
import pickle
import transformers
from transformers import AutoTokenizer 
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import pipeline
from scipy.special import softmax

In [2]:
# Requirements
model_path ="HOLYBOY/Sentiment_Analysis"
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [3]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = "@user" if t.startswith("@") and len(t) > 1 else t
        t = "http" if t.startswith("http") else t
        new_text.append(t)
    return " ".join(new_text)

In [4]:
# ---- Function to process the input and return prediction
def sentiment_analysis(text):
    text = preprocess(text)

    encoded_input = tokenizer(text, return_tensors = "pt") # for PyTorch-based models
    output = model(**encoded_input)
    scores_ = output[0][0].detach().numpy()
    scores_ = softmax(scores_)
    
    # Format output dict of scores
    labels = ["Negative", "Neutral", "Positive"]
    scores = {l:float(s) for (l,s) in zip(labels, scores_) }
    
    return scores

In [10]:
# ---- Gradio app interface
app = gr.Interface(fn = sentiment_analysis,
                   inputs = gr.Textbox("Input your tweet to classify or use the example provided below..."),
                   outputs = "label",
                   title = "Public Perception of COVID-19 Vaccines",
                   description  = "This app analyzes Perception of text based on tweets about COVID-19 Vaccines using a fine-tuned distilBERT model",
                   interpretation = "default",
                   examples = [["The idea of introducing the vaccine is good"],
                               ["I am definately not taking the jab"], 
                               ["The vaccine is bad and can cause serious health implications"], 
                               ["I dont have any opinion "]]
                   )

app.launch(share =True)

Running on local URL:  http://127.0.0.1:7865

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app. 

Also please ensure that your antivirus or firewall is not blocking the binary file located at: C:\Users\user\anaconda3\lib\site-packages\gradio\frpc_windows_amd64_v0.2
